# Dynamic Visualization of Bornholdt Simulation Datasets

This notebook provides an interactive 3D visualization of all simulation results stored in the `Sim_Datasets/` directory.  
It automatically scans available `.npz` files, extracts their parameter combinations $(J, \alpha, \beta, \text{Seed})$, and displays them in a dynamic 3D scatter plot.  
Users can filter datasets by parameter ranges or seeds, color points by different parameters, and inspect file metadata directly through hover text.  

A background watcher monitors `Sim_Datasets/` for changes and refreshes the visualization whenever new simulations are added or removed, ensuring the plot always reflects the current dataset state.


In [1]:
# -----------------------------
# Imports
# -----------------------------

import re
import os
from pathlib import Path

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

import ipywidgets as W
from IPython.display import display

In [2]:
"""
Dynamic 3D view of available simulation datasets in Sim_Datasets/
- Scans filenames like: bornholdt_simulation_J-1.0_Alpha-6.0_Beta-0.8_Seed-42.npz
- Parses (J, Alpha, Beta, Seed)
- Shows an interactive 3D scatter: x=J, y=Beta, z=Alpha
- Widgets to filter by seed and parameter ranges
- Hover shows file metadata (including N, STEPS, BURN_IN if present in .npz)
"""

# -----------------------------
# Locate Sim_Datasets dir
# -----------------------------
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path.cwd()

SIM_DIR = BASE_DIR / "Sim_Datasets"
SIM_DIR.mkdir(exist_ok=True)

# -----------------------------
# Filename parser
# -----------------------------
# Accept integers or decimals with exactly one decimal (your fmt1), but be tolerant.
PATTERN = re.compile(
    r"bornholdt_simulation_"
    r"J-(?P<J>[0-9]+(?:\.[0-9]+)?)_"
    r"Alpha-(?P<Alpha>[0-9]+(?:\.[0-9]+)?)_"
    r"Beta-(?P<Beta>[0-9]+(?:\.[0-9]+)?)_"
    r"Seed-(?P<Seed>[0-9]+)\.npz$"
)

def scan_sim_dir(sim_dir: Path) -> pd.DataFrame:
    rows = []
    for p in sim_dir.glob("bornholdt_simulation_*.npz"):
        m = PATTERN.search(p.name)
        if not m:
            continue
        J = float(m.group("J"))
        Alpha = float(m.group("Alpha"))
        Beta = float(m.group("Beta"))
        Seed = int(m.group("Seed"))
        # Try to pull metadata from the file (optional, robust to missing keys)
        N = STEPS = BURN_IN = None
        try:
            with np.load(p, allow_pickle=False) as z:
                N = int(z["N"]) if "N" in z.files else None
                STEPS = int(z["STEPS"]) if "STEPS" in z.files else None
                BURN_IN = int(z["BURN_IN"]) if "BURN_IN" in z.files else None
        except Exception:
            pass
        rows.append({
            "J": J, "Alpha": Alpha, "Beta": Beta, "Seed": Seed,
            "path": str(p),
            "filename": p.name,
            "N": N, "STEPS": STEPS, "BURN_IN": BURN_IN,
            "size_kb": round(p.stat().st_size / 1024, 1)
        })
    df = pd.DataFrame(rows)
    if not df.empty:
        df.sort_values(["J", "Alpha", "Beta", "Seed"], inplace=True)
    return df

df_all = scan_sim_dir(SIM_DIR)

if df_all.empty:
    print(f"No .npz files found in {SIM_DIR}.")
else:
    # -----------------------------
    # Widgets
    # -----------------------------
    seed_options = ["All"] + sorted(df_all["Seed"].unique().tolist())
    seed_dd = W.Dropdown(options=seed_options, value="All", description="Seed:", layout=W.Layout(width="220px"))

    j_min, j_max = df_all["J"].min(), df_all["J"].max()
    a_min, a_max = df_all["Alpha"].min(), df_all["Alpha"].max()
    b_min, b_max = df_all["Beta"].min(), df_all["Beta"].max()

    j_range = W.FloatRangeSlider(value=[j_min, j_max], min=j_min, max=j_max, step=0.1, description="J",
                                 layout=W.Layout(width="420px"), readout_format=".1f")
    a_range = W.FloatRangeSlider(value=[a_min, a_max], min=a_min, max=a_max, step=0.1, description="α",
                                 layout=W.Layout(width="420px"), readout_format=".1f")
    b_range = W.FloatRangeSlider(value=[b_min, b_max], min=b_min, max=b_max, step=0.1, description="β",
                                 layout=W.Layout(width="420px"), readout_format=".1f")

    color_by = W.Dropdown(options=["Beta", "Alpha", "J", "Seed"], value="Beta", description="Color:",
                          layout=W.Layout(width="220px"))
    size_by = W.Dropdown(options=["constant", "size_kb"], value="constant", description="Size:",
                         layout=W.Layout(width="220px"))
    marker_size = W.IntSlider(value=6, min=3, max=14, step=1, description="Marker",
                              layout=W.Layout(width="300px"))

    # -----------------------------
    # Figure builder
    # -----------------------------
    def make_hover_text(sub: pd.DataFrame) -> pd.Series:
        # Compact hover with key metadata
        def one(row):
            meta = []
            meta.append(f"J={row['J']:.1f}, α={row['Alpha']:.1f}, β={row['Beta']:.1f}")
            meta.append(f"Seed={row['Seed']}")
            if pd.notna(row.get("N", None)): meta.append(f"N={int(row['N'])}")
            if pd.notna(row.get("STEPS", None)): meta.append(f"STEPS={int(row['STEPS'])}")
            if pd.notna(row.get("BURN_IN", None)): meta.append(f"BURN_IN={int(row['BURN_IN'])}")
            meta.append(row["filename"])
            return "<br>".join(meta)
        return sub.apply(one, axis=1)

    def filter_df():
        sub = df_all.copy()
        # seed filter
        if seed_dd.value != "All":
            sub = sub[sub["Seed"] == seed_dd.value]
        # range filters
        j_lo, j_hi = j_range.value
        a_lo, a_hi = a_range.value
        b_lo, b_hi = b_range.value
        sub = sub[(sub["J"] >= j_lo) & (sub["J"] <= j_hi) &
                  (sub["Alpha"] >= a_lo) & (sub["Alpha"] <= a_hi) &
                  (sub["Beta"] >= b_lo) & (sub["Beta"] <= b_hi)]
        return sub

    def build_fig():
        sub = filter_df()
        if sub.empty:
            fig = go.Figure()
            fig.update_layout(height=640, title="No points in current filter", scene=dict(
                xaxis_title="J (herding strength)",
                yaxis_title="β (inverse temperature)",
                zaxis_title="α (contrarian strength)"
            ))
            return fig

        hover = make_hover_text(sub)
        if size_by.value == "size_kb":
            sizes = np.clip((sub["size_kb"].values / max(sub["size_kb"].max(), 1.0)) * 12 + 4, 4, 18)
        else:
            sizes = np.full(len(sub), marker_size.value)

        fig = px.scatter_3d(
            sub,
            x="J", y="Beta", z="Alpha",
            color=color_by.value,
            size=None,  # we handle size manually for better control
            title="Simulation Dataset Coverage (Sim_Datasets)",
            height=640
        )
        # Apply custom sizes and hover text
        fig.update_traces(marker=dict(size=sizes, line=dict(width=0.5, color="black")), hovertext=hover, hoverinfo="text")
        fig.update_layout(
            scene=dict(
                xaxis_title="J (herding)",
                yaxis_title="β (inverse temp.)",
                zaxis_title="α (contrarian)"
            ),
            legend_title_text=color_by.value
        )
        return fig

    out_fig = W.Output()

    def refresh(_=None):
        with out_fig:
            out_fig.clear_output(wait=True)
            fig = build_fig()
            fig.show()

    # Hook up callbacks
    for w in [seed_dd, j_range, a_range, b_range, color_by, size_by, marker_size]:
        w.observe(refresh, names="value")

    # Initial render
    controls_top = W.HBox([seed_dd, color_by, size_by, marker_size])
    controls_ranges = W.HBox([j_range, b_range, a_range])  # order to match axes x(J), y(β), z(α)
    display(W.VBox([controls_top, controls_ranges, out_fig]))
    refresh()
